In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131323 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
!mkdir -p drive

In [0]:
!google-drive-ocamlfuse drive

In [0]:
!pip install keras
!pip install h5py

In [0]:
!ls drive

In [0]:
import torch
import torchvision
torch.cuda.is_available()

True

In [0]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import pandas as pd
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset

In [0]:
data=pd.read_csv('drive/Fast-Pytorch/Learning_Pytorch/datasets/diabetes.csv') # Colad file path
x=data.iloc[:,0:-1].values # all rows, first to last column (except last column), then convert numpy array with .values
y_string=list(data.iloc[:,-1]) # convert to list only last column

In [0]:
y_int=[]
for string in y_string:
  if string=='positive':
    y_int.append(1)
  else:
    y_int.append(0)
y= np.array(y_int, dtype='float64')

In [0]:
sc=StandardScaler() # normalization, x-mean/std
x=sc.fit_transform(x) # apply to data
x=torch.tensor(x) # convert numpy to torch tensor
y=torch.tensor(y).unsqueeze(1) # add extra dim (768,) to (768,1)

In [0]:
print(x.shape,x.dtype)
print(y.shape,y.dtype)

torch.Size([768, 7]) torch.float64
torch.Size([768, 1]) torch.float64


In [0]:
x=Variable(x) #Variable initialization
y=Variable(y)

In [0]:
class Dataset(Dataset):
  def __init__(self,x,y):
    self.x=x
    self.y=y
  
  def __getitem__(self,index):
    return self.x[index], self.y[index]
  
  def __len__(self):
    return len(self.x)

In [0]:
dataset=Dataset(x,y)
print(dataset.x.shape)
print(dataset.y.shape)

torch.Size([768, 7])
torch.Size([768, 1])


In [0]:
train_loader= torch.utils.data.DataLoader(dataset=dataset, batch_size=32, shuffle=True) # load data

In [0]:
class Model(nn.Module):
  def __init__(self):
    super(Model,self).__init__()
    self.fc1 =torch.nn.Linear(x.shape[1],5)
    self.fc2 =torch.nn.Linear(5,3)
    self.fc3 =torch.nn.Linear(3,1)
    self.sigmoid=torch.nn.Sigmoid()
    
  def forward(self,x):
    out =self.fc1(x)
    out =self.sigmoid(out)
    out =self.fc2(out)
    out =self.sigmoid(out)
    out =self.fc3(out)
    out= self.sigmoid(out)
    return out

In [0]:
net = Model()
criterion =torch.nn.BCELoss(size_average=True)
optimizer =torch.optim.SGD(net.parameters(), lr=0.1)

/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [0]:
num_epochs =1000
for epoch in range(num_epochs):
  for inputs, labels in train_loader:
    inputs = Variable(inputs.float())
    labels= Variable(labels.float())
    output = net(inputs)
    optimizer.zero_grad()
    loss = criterion(output,labels)
    # loss = -(labels * torch.log(output)+ (1-labels) * torch.log(1-output)).mean()
    loss.backward()
    optimizer.step()
  output = (output>0.5).float()
  correct = (output == labels).float().sum()
  print("Epoch {}/{}, loss:{:.3f}, Accuracy: {:.3f}".format(epoch+1,num_epochs,loss.item(),correct/output.shape[0]))

Epoch 1/1000, loss:0.680, Accuracy: 0.594
Epoch 2/1000, loss:0.644, Accuracy: 0.656
Epoch 3/1000, loss:0.586, Accuracy: 0.750
Epoch 4/1000, loss:0.722, Accuracy: 0.531
Epoch 5/1000, loss:0.764, Accuracy: 0.469
Epoch 6/1000, loss:0.662, Accuracy: 0.625
Epoch 7/1000, loss:0.645, Accuracy: 0.656
Epoch 8/1000, loss:0.642, Accuracy: 0.656
Epoch 9/1000, loss:0.566, Accuracy: 0.781
Epoch 10/1000, loss:0.584, Accuracy: 0.750
Epoch 11/1000, loss:0.662, Accuracy: 0.625
Epoch 12/1000, loss:0.623, Accuracy: 0.688
Epoch 13/1000, loss:0.680, Accuracy: 0.594
Epoch 14/1000, loss:0.682, Accuracy: 0.594
Epoch 15/1000, loss:0.508, Accuracy: 0.875
Epoch 16/1000, loss:0.622, Accuracy: 0.688
Epoch 17/1000, loss:0.723, Accuracy: 0.531
Epoch 18/1000, loss:0.702, Accuracy: 0.562
Epoch 19/1000, loss:0.604, Accuracy: 0.719
Epoch 20/1000, loss:0.745, Accuracy: 0.500
Epoch 21/1000, loss:0.526, Accuracy: 0.844
Epoch 22/1000, loss:0.681, Accuracy: 0.594
Epoch 23/1000, loss:0.601, Accuracy: 0.719
Epoch 24/1000, loss:

In [0]:
num_epochs =1000
for epoch in range(num_epochs):
  for inputs, labels in train_loader:
    inputs = Variable(inputs.float())
    labels= Variable(labels.float())
    output = net(inputs)
    optimizer.zero_grad()
    loss = criterion(output,labels)
    # loss = -(labels * torch.log(output)+ (1-labels) * torch.log(1-output)).mean()
    loss.backward()
    optimizer.step()
  output = (output>0.5).float()
  accuracy = (output == labels).float().mean()
  print("Epoch {}/{}, loss:{:.3f}, Accuracy: {:.3f}".format(epoch+1,num_epochs,loss.item(),accuracy))

Epoch 1/1000, loss:0.384, Accuracy: 0.812
Epoch 2/1000, loss:0.329, Accuracy: 0.844
Epoch 3/1000, loss:0.548, Accuracy: 0.750
Epoch 4/1000, loss:0.371, Accuracy: 0.812
Epoch 5/1000, loss:0.374, Accuracy: 0.844
Epoch 6/1000, loss:0.299, Accuracy: 0.875
Epoch 7/1000, loss:0.332, Accuracy: 0.812
Epoch 8/1000, loss:0.539, Accuracy: 0.750
Epoch 9/1000, loss:0.420, Accuracy: 0.750
Epoch 10/1000, loss:0.301, Accuracy: 0.938
Epoch 11/1000, loss:0.736, Accuracy: 0.625
Epoch 12/1000, loss:0.314, Accuracy: 0.875
Epoch 13/1000, loss:0.304, Accuracy: 0.875
Epoch 14/1000, loss:0.346, Accuracy: 0.844
Epoch 15/1000, loss:0.583, Accuracy: 0.656
Epoch 16/1000, loss:0.349, Accuracy: 0.812
Epoch 17/1000, loss:0.460, Accuracy: 0.750
Epoch 18/1000, loss:0.315, Accuracy: 0.844
Epoch 19/1000, loss:0.538, Accuracy: 0.750
Epoch 20/1000, loss:0.608, Accuracy: 0.594
Epoch 21/1000, loss:0.551, Accuracy: 0.719
Epoch 22/1000, loss:0.345, Accuracy: 0.875
Epoch 23/1000, loss:0.381, Accuracy: 0.750
Epoch 24/1000, loss: